<a href="https://colab.research.google.com/github/iamluca17/WIDS2025_MDLV/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0

In [5]:
pip install numpy pandas pyarrow openpyxl fastparquet matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should print True if GPU is enabled
print(torch.cuda.get_device_name(0))  # Should show the GPU model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor = torch.randn(3, 3).to(device)
print(tensor.device)  # Should print 'cuda:0' if GPU is used

2.4.0+cu121
True
Tesla T4
cuda:0


In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
  Using cached https://data.pyg.org/whl/torch-2.4.0%2Bcu121/torch_scatter-2.1.2%2Bpt24cu121-cp311-cp311-linux_x86_64.whl (10.9 MB)
  Using cached https://data.pyg.org/whl/torch-2.4.0%2Bcu121/torch_sparse-0.6.18%2Bpt24cu121-cp311-cp311-linux_x86_64.whl (5.1 MB)
  Using cached https://data.pyg.org/whl/torch-2.4.0%2Bcu121/torch_cluster-1.6.3%2Bpt24cu121-cp311-cp311-linux_x86_64.whl (3.4 MB)
  Using cached https://data.pyg.org/whl/torch-2.4.0%2Bcu121/torch_spline_conv-1.2.2%2Bpt24cu121-cp311-cp311-linux_x86_64.whl (989 kB)
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [6]:
import numpy as np
import pandas as pd

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_add_pool,  global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [7]:
def adj_mat_preproc(M):

  M += M.T
  M += np.eye(len(M))

  return M

def gen_graph(adj_mat, label):

  A = adj_mat
  edge_indices = (A > 0).nonzero(as_tuple = False).t()
  edge_attr = A[edge_indices[0], edge_indices[1]]
  X = torch.eye(len(A))

  graph_data = Data(x=X, edge_index=edge_indices, edge_attr=edge_attr, y=label)
  return graph_data

def gen_graph_dataset(data_matrices, labels):

  adj_matrices = torch.tensor(data_matrices).float
  labels = torch.tensor(labels).float

  graph_dataset = []

  for i in range(len(adj_matrices)):


    A = adj_matrices[i]
    label = labels[i]
    graph_data = gen_graph(A, label)

    graph_dataset.append(graph_data)

  return graph_dataset


In [ ]:
class GCN(torch.nn.Module):

  def __init__(self):
    super(GCN,self).__init__()
    self.conv1 = GCNConv(in_channels=200, out_channels=200)
    self.conv2 = GCNConv(in_channels=200, out_channels=200)


  def forward(self, data):

    x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

    x = self.conv1(x, edge_index, edge_attr)
    x = F.relu(x)

    x = self.conv2(x, edge_index, edge_attr)

    x = global_mean_pool(x, batch)
